## **Hotspot Prediction Tool (Trinity)**
##### This hotspot prediction tool is specifically designed to analyze genomic sequences, generating all potential secondary structures within a specified range of 50 to 200 base pairs. It then identifies the structure with the lowest free energy, indicative of a potential hotspot-related RNA-interference (RNAi) precursor. The input sequences typically originate from experimental data, such as small RNA sequencing studies on viruses, aimed at identifying regions of the viral genome targeted by the host RNAi machinery.

0. Starting line to import and install of all the needed packages

In [1]:
pip install biopython

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install viennarna

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pillow

Note: you may need to restart the kernel to use updated packages.


1. Computation of Hotspot Subsequences

In [5]:
from Bio import SeqIO

def generate_subsequences(sequence, min_length=21):
    subsequences = []
    for i in range(len(sequence) - min_length + 1):
        for j in range(min_length, len(sequence) - i + 1):
            subsequences.append(sequence[i:i+j])
    return subsequences

def write_subsequences_to_fasta(sequence_id, subsequences, output_folder):
    with open(f"{output_folder}/{sequence_id}_subsequences.fasta", "w") as output_file:
        for i, subsequence in enumerate(subsequences):
            output_file.write(f">{sequence_id}_subsequence_{i+1}\n{subsequence}\n")

def main(input_fasta, output_folder):
    for record in SeqIO.parse(input_fasta, "fasta"):
        subsequences = generate_subsequences(str(record.seq))
        write_subsequences_to_fasta(record.id, subsequences, output_folder)

if __name__ == "__main__":
    input_fasta = "GRBV_Hotspots.fasta"  # Path to input FASTA file
    output_folder = "subsequences_output"  # Output folder to save subsequence FASTA files
    main(input_fasta, output_folder)

2. Secondary Structure Prediction and MFE Ranking (using Hotspot 1 as example)

In [1]:
import os
from Bio import SeqIO
import RNA

def predict_secondary_structure(sequence):
    # Initialize ViennaRNA RNAfold
    (ss, mfe) = RNA.fold(sequence)
    return ss, mfe

def rank_subsequences(input_fasta, output_folder):
    lowest_mfe = float('inf')  # Initialize with a large value
    lowest_mfe_sequence = None
    lowest_mfe_structure = None
    ranked_subsequences = []
    
    # Count the total number of sequences in the input FASTA file
    total_sequences = sum(1 for _ in SeqIO.parse(input_fasta, "fasta"))
    processed_sequences = 0

    print("Starting the analysis...")
    
    # Iterate through each sequence in the input FASTA file
    for record in SeqIO.parse(input_fasta, "fasta"):
        processed_sequences += 1
        
        # Print progress update after the 10th analyzed sequence
        if processed_sequences == 10:
            print("Analysis in progress...")
        
        # Print progress update every 1000 sequences analyzed
        if processed_sequences % 100 == 0:
            print(f"Processed {processed_sequences} out of {total_sequences} sequences")
            
            # Print progress update every 1000 sequences analyzed
        if processed_sequences == total_sequences:
            print("Analysis Completed!")
        
        sequence = str(record.seq)
        
        # Iterate through all possible subsequences of the sequence
        for i in range(len(sequence)):
            for j in range(i + 50, min(i + 400, len(sequence))):  # Limiting to subsequences of length 50 to 400
                subsequence = sequence[i:j]
                
                # Predict the secondary structure and minimum free energy (MFE) using RNAfold
                ss, mfe = predict_secondary_structure(subsequence)
                ranked_subsequences.append((subsequence, mfe))
                
                # Update lowest MFE sequence and structure if found
                if mfe < lowest_mfe:
                    lowest_mfe = mfe
                    lowest_mfe_sequence = subsequence
                    lowest_mfe_structure = ss

    # Sort the ranked subsequences based on MFE (ascending order)
    ranked_subsequences.sort(key=lambda x: x[1])
    
    # Save the top 20 subsequences with the lowest MFE to a file
    with open(os.path.join(output_folder, "ranked_subsequences.txt"), "w") as output_file:
        output_file.write("Subsequence\tMinimum Free Energy\n")
        for subsequence, mfe in ranked_subsequences[:20]:
            output_file.write(f"{subsequence}\t{mfe}\n")
    
    # Write the sequence with the lowest MFE to a separate file
    if lowest_mfe_sequence is not None:
        lowest_mfe_length = len(lowest_mfe_sequence)
        with open(os.path.join(output_folder, "sequence_with_lowest_mfe.txt"), "w") as lowest_mfe_file:
            lowest_mfe_file.write(f"Sequence with lowest MFE:\n{lowest_mfe_sequence}\nLength: {lowest_mfe_length}\nMFE: {lowest_mfe}")
        
        # Save the secondary structure to a dot file
        dot_file_path = os.path.join(output_folder, "lowest_mfe_structure.dot")
        with open(dot_file_path, "w") as dot_file:
            dot_file.write(f">{lowest_mfe_sequence}\n{lowest_mfe_structure}")


if __name__ == "__main__":
    input_fasta = "subsequences_output/Hotspot_1_subsequences.fasta"  # Path to the input FASTA file containing the subsequences
    output_folder = "ranked_subsequences_output_1"  # Output folder to save the ranked subsequences
    rank_subsequences(input_fasta, output_folder)

Starting the analysis...
Analysis in progress...
Processed 100 out of 8256 sequences
Processed 200 out of 8256 sequences
Processed 300 out of 8256 sequences
Processed 400 out of 8256 sequences
Processed 500 out of 8256 sequences
Processed 600 out of 8256 sequences
Processed 700 out of 8256 sequences
Processed 800 out of 8256 sequences
Processed 900 out of 8256 sequences
Processed 1000 out of 8256 sequences
Processed 1100 out of 8256 sequences
Processed 1200 out of 8256 sequences
Processed 1300 out of 8256 sequences
Processed 1400 out of 8256 sequences
Processed 1500 out of 8256 sequences
Processed 1600 out of 8256 sequences
Processed 1700 out of 8256 sequences
Processed 1800 out of 8256 sequences
Processed 1900 out of 8256 sequences
Processed 2000 out of 8256 sequences
Processed 2100 out of 8256 sequences
Processed 2200 out of 8256 sequences
Processed 2300 out of 8256 sequences
Processed 2400 out of 8256 sequences
Processed 2500 out of 8256 sequences
Processed 2600 out of 8256 sequence

FileNotFoundError: [Errno 2] No such file or directory: 'ranked_subsequences_output_1/ranked_subsequences.txt'

3. For each candidate sequence, estimate the k-mers ranging from 50 to 250 bp, on which the structure prediction will be performed

*Notebook Created By: Christian Mandelli, Oregon State University*